In [4]:
# we import all the libraries we are going to use
# we have to use selinium, since our web page is dynamic
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import time
import csv

#this function if to click on the "show more button" more than once, with sleep time
def click_show_more(driver, max_attempts, delay_between_attempts):
    attempts = 0
    while attempts < max_attempts:
        try:
            # Wait for button to be present and clickable
            show_more_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, "//button[contains(., 'Show more')]"))
            )
            
            # Scroll button into view and click
            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", show_more_button)
            driver.execute_script("arguments[0].click();", show_more_button)
            
            print(f"Successfully clicked 'Show more' (attempt {attempts + 1})")
            
            # Wait for new content to load
            time.sleep(delay_between_attempts)
            
        except Exception as e:
            print(f"Stopping - couldn't find/click 'Show more' (attempt {attempts + 1}): {str(e)}")
            break
            
        attempts += 1

# with this function, we will scap al the reviews
def scrape_reviews(url):
    # Initialize WebDriver
    driver = webdriver.Chrome()
    driver.get(url)

    # Wait for initial content
    WebDriverWait(driver, 15).until(
        EC.presence_of_element_located((By.CLASS_NAME, "kl_reviews__review_item"))
    )

    # Get initial count of reviews
    # initial_count = len(driver.find_elements(By.CLASS_NAME, "kl_reviews__review_item"))
    # print(f"Initial reviews loaded: {initial_count}")

    # We call the show more function from before
    click_show_more(driver, max_attempts=3, delay_between_attempts=2)

    # Verify final count
    final_count = len(driver.find_elements(By.CLASS_NAME, "kl_reviews__review_item"))
    print(f"Total reviews after clicking: {final_count}")

    # Parse and collect results
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    review_items = soup.find_all(class_="kl_reviews__review_item")

    reviews = []

    for i, review in enumerate(review_items, 1):
        title = review.find(class_="kl_reviews__review__title")
        title_text = title.get_text(strip=True) if title else "No title"
        
        content = review.find(class_="kl_reviews__review__content")
        content_text = content.get_text(strip=True) if content else "No content"
        
        reviews.append({
            "Review": f"Review {i}",
            "Title": title_text,
            "Content": content_text,
        })

    driver.quit()
    return reviews

def save_to_csv(reviews, filename='reviews.csv'):
    """Save reviews to a CSV file"""
    with open(filename, 'w', newline='', encoding='utf-8') as csvfile:
        fieldnames = ['Review', 'Title', 'Content']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames, delimiter="|")
        
        writer.writeheader()
        writer.writerows(reviews)
    
    print(f"Successfully saved {len(reviews)} reviews to {filename}")

if __name__ == "__main__":
    url = "https://es.esn.com/en/products/esn-isoclear-whey-isolate?variant=54000126460172"
    reviews = scrape_reviews(url)
    save_to_csv(reviews)

Successfully clicked 'Show more' (attempt 1)
Successfully clicked 'Show more' (attempt 2)
Successfully clicked 'Show more' (attempt 3)
Total reviews after clicking: 20
Successfully saved 20 reviews to reviews.csv
